<a href="https://colab.research.google.com/github/chatchawan90/CMX-Analytics/blob/revenue_with_cat/Copy_of_Merck_Revenue_with_Categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%sh
apt --fix-broken install

Reading package lists...
Building dependency tree...
Reading state information...
Correcting dependencies... Done
The following packages will be REMOVED:
  msodbcsql17 unixodbc
0 upgraded, 0 newly installed, 2 to remove and 30 not upgraded.
2 not fully installed or removed.
After this operation, 114 kB disk space will be freed.
(Reading database ... 120526 files and directories currently installed.)
Removing msodbcsql17 (17.10.4.1-1) ...
/var/lib/dpkg/info/msodbcsql17.postrm: line 11: odbcinst: command not found
dpkg: error processing package msodbcsql17 (--remove):
 installed msodbcsql17 package post-removal script subprocess returned error exit status 127
dpkg: too many errors, stopping
Errors were encountered while processing:
 msodbcsql17
Processing was halted because there were too many errors.




E: Sub-process /usr/bin/dpkg returned an error code (1)


CalledProcessError: ignored

In [ ]:
%%sh
odbcinst -j

sh: 1: odbcinst: not found


CalledProcessError: ignored

In [ ]:
%%sh

sudo apt --fix-broken install
sudo apt-get update
sudo apt-get upgrade

Reading package lists...
Building dependency tree...
Reading state information...
Correcting dependencies... Done
The following packages will be REMOVED:
  msodbcsql17 unixodbc
0 upgraded, 0 newly installed, 2 to remove and 30 not upgraded.
2 not fully installed or removed.
After this operation, 114 kB disk space will be freed.
(Reading database ... 120509 files and directories currently installed.)
Removing msodbcsql17 (17.10.4.1-1) ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78.)
debconf: falling back to frontend: Readline
/var/lib/dpkg/info/msodbcsql17.postrm: line 11: odbcinst: command not found
dpkg: error processing package msodbcsql17 (--remove):
 installed msodbcsql17 package post-removal script subprocess returned error exit status 127
dpkg: too many errors, stopping
Errors were encountered while processing:
 msodbcsql17




E: Sub-process /usr/bin/dpkg returned an error code (1)
W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
E: Unmet dependencies. Try 'apt --fix-broken install' with no packages (or specify a solution).


CalledProcessError: ignored

In [ ]:
%%sh

curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -

curl https://packages.microsoft.com/config/ubuntu/22.04/prod.list > /etc/apt/sources.list.d/mssql-release.list

sudo apt-get update

sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17



OK
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease [3,611 B]
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main amd64 Packages [82.6 kB]
Get:7 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main all Packages [970 B]
Get:8 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main armhf Packages [8,639 B]
Get:9 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main arm64 Packages [17.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   983  100   983    0     0   4720      0 --:--:-- --:--:-- --:--:--  4703
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0    593      0 --:--:-- --:--:-- --:--:--   594
W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/

In [ ]:
import pandas as pd
import numpy as np
import pyodbc
import datetime
import openpyxl
from dateutil.relativedelta import relativedelta

In [ ]:
!pip install pyodbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.5/343.5 kB 8.6 MB/s eta 0:00:00


In [ ]:
server = 'sichemex.fortiddns.com,1444'
database = 'dbwins_cmx'
username = 'chemex'
password = '6vQ~cDx6yCpP(CQ`'


In [ ]:

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()


In [ ]:
%%sh
lsb_release -a

Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


No LSB modules are available.


In [ ]:

def retreive_val ( query):
    """execute query and turn the data in the a list of dict

    Args:
        query (_type_): _description_

    Returns:
        list of dictionary: all data from the data as a list of dict
    """
    cursor.execute(query)
    result = cursor.fetchall()
    col = [column[0] for column in cursor.description]
    result_LoD = [dict(zip(col,x)) for x in result ]
    # LoD = list of dict
    return col,result_LoD

In [ ]:
revenue_pd = pd.DataFrame(retreive_val("select * from bi_revenue where docuno like 'IV%'")[1])

In [ ]:
# If file not found, copy the file path from the panel on the left (assuming you are on GDrive)
cat_pd = pd.read_csv('/content/gdrive/MyDrive/Chemical Express/Marketing Google Drive/Analytics/Merck Product Categories Extraction/Data/CSV/cleaned_unique_category.csv',encoding = 'unicode_escape')

In [ ]:
import numpy as np
from google.colab import autoviz
df_1153411048952213344 = autoviz.get_df('df_1153411048952213344')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_1153411048952213344, *['business_unit_desc'], **{})
chart

In [ ]:
import numpy as np
from google.colab import autoviz
df_1153411048952213344 = autoviz.get_df('df_1153411048952213344')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_1153411048952213344, *['business_unit_code'], **{})
chart

In [ ]:
revenue_pd_dict = revenue_pd.to_dict('records')
cat_pd_dict = cat_pd.to_dict('records')

In [ ]:
cat_pd_dict[0]

{'product_code': '100020.1000',
 'product_description': 'ACETONE FOR LIQUID CHROMATOGRAPHY LICHRO',
 'sbu_code': '526',
 'sbu_description': 'Solvents Instrumental Analysis',
 'business_field_desc': 'Analytical Chemistry & Solvents',
 'business_field_code': 'C45',
 'business_unit_code': 'LR2',
 'business_unit_desc': 'Chemistry'}

In [ ]:
revenue_pd_dict[0]

{'docuno': 'IVZ6412-00242',
 'docudate': datetime.datetime(2021, 12, 7, 0, 0),
 'บริษัทหรือหน่วยงาน': 'สถาบันวิทยสิริเมธี',
 'รหัสลูกค้า': 'ZH-VIS-02',
 'ลูกค้า': 'คุณอภิสิทธิ์ .',
 'ชื่อที่อยู่บิล': 'สถาบันวิทยสิริเมธี',
 'Poหรือเอกสารอ้างอิง': None,
 'ลำดับ': 1,
 'แบรนด์': 'TCI',
 'รหัสสินค้า': 'TCI-D0574-25G',
 'รายละเอียดสินค้า': '2,5-DIHYDROXY-1,4-BENZOQUINONE "TCI" 25G',
 'vatgroup': 'ไม่รวมvatในสินค้า',
 'ส่วนลด': Decimal('0.0000'),
 'ยอดมูลค่าขายและลดหนี้ก่อนVAT': Decimal('5070.000000'),
 'ราคาต่อหน่วยก่อนVAT': Decimal('5070.000000'),
 'หน่วยนับ': 'Bott',
 'จำนวนขาย': Decimal('1.0000'),
 'Sales': 'เสาวภา  ชโลทร  (T.086-448-6508)',
 'ชนิดผู้ขาย': 'Sales',
 'บิลปลอม': ''}

In [ ]:
isinstance('s', list)

False

In [ ]:
'k-i-a'.split('-')

['k', 'i', 'a']

In [ ]:
for r in revenue_pd_dict:
  r['found'] = False
  r['รหัสสินค้า'] = r['รหัสสินค้า'].upper()
  for c in cat_pd_dict:
    if isinstance(c['product_code'], str):
      if r['รหัสสินค้า'].upper() == c['product_code'].upper():
        for k,v in c.items():
          if k != 'product_code' and k != 'product_description':
            r[k] = c[k]
            r['found']= True
      else:
        if 'SIA' in r['รหัสสินค้า'].upper() and r['รหัสสินค้า'].count('-')==2:
           if r['รหัสสินค้า'].split('-')[1] in  c['product_code'].upper():
            for k,v in c.items():
              if k != 'product_code' and k != 'product_description':
                r[k] = c[k]
                r['found']= True
    else:
      cap_list = [x.upper() for x in c['product_code']]

      if r['รหัสสินค้า'].upper() in cap_list:
        for k,v in c.items():
          if k != 'product_code' and k != 'product_description':
            r[k] = c[k]
            r['found']= True
      else:
        if 'SIA' in r['รหัสสินค้า'].upper() and r['รหัสสินค้า'].count('-')==2:
           if r['รหัสสินค้า'].split('-')[1] in  cap_list:
            for k,v in c.items():
              if k != 'product_code' and k != 'product_description':
                r[k] = c[k]
                r['found']= True


# New Section

In [ ]:
searched = pd.DataFrame(revenue_pd_dict)
merck_brand = ['Sigma-Aldrich', 'Supelco', 'Roche', 'Sigma', 'Millipore', 'Aldrich', 'Supelco', 'Merck']
searched= searched[searched['แบรนด์'].isin(merck_brand)]

In [ ]:
searched.drop(columns=['product_code', 'product_description'])

,docuno,docudate,บริษัทหรือหน่วยงาน,รหัสลูกค้า,ลูกค้า,ชื่อที่อยู่บิล,Poหรือเอกสารอ้างอิง,ลำดับ,แบรนด์,รหัสสินค้า,...,Sales,ชนิดผู้ขาย,บิลปลอม,found,sbu_code,sbu_description,business_field_desc,business_field_code,business_unit_code,business_unit_desc
7,IVZ6411-00316,2021-11-29 00:00:00,บริษัท ศูนย์พันธุศาสตร์การแพทย์ จำกัด,ZH-MGC,คุณรุจนาถ,บริษัท ศูนย์พันธุศาสตร์การแพทย์ จำกัด,None,1,Merck,104873.1000,...,เสาวภา ชโลทร (T.086-448-6508),Sales,,True,702,Classical Inorganics for QC,Analytical Chemistry & Solvents,C45,LR2,Chemistry
10,IVZ6411-00323,2021-11-29 00:00:00,สถาบันวิจัยจุฬาภรณ์,ZH-CRI,คุณอภินันท์ คณะสุวรรณ์,ราชวิทยาลัยจุฬาภรณ์,None,1,Merck,6500022854,...,เสาวภา ชโลทร (T.086-448-6508),Sales,,False,NaN,NaN,NaN,NaN,NaN,NaN
12,IVZ6411-00325,2021-11-29 00:00:00,คณะอุตสาหกรรมเกษตร มหาวิทยาลัยเกษตรศาสตร์,ZH-KU-AGR,คุณกานต์กนก,โครงการวิจัยแนวทางใหม่ในการลดความเสี่ยงจากการไ...,None,1,Merck,111972.0500,...,วิรงรอง ทักษเวทย์ (T.089-455-9181),Sales,,False,NaN,NaN,NaN,NaN,NaN,NaN
13,IVZ6411-00326,2021-11-29 00:00:00,ภาควิชาจุลชีววิทยา คณะวิทยาศาสตร์ มหาวิทยาลัยม...,ZH-MU-SC-MIC,คุณณัฐณิชา,scDengue Newton/NSTDA หน่วยวิจัยชีววิทยาเชิงร...,None,1,Merck,06-727,...,เสาวภา ชโลทร (T.086-448-6508),Sales,,False,NaN,NaN,NaN,NaN,NaN,NaN
14,IVZ6411-00328,2021-11-29 00:00:00,มหาวิทยาลัยราชภัฎสวนสุนันทา,RI-SS-005,คุณสุวิมล,มหาวิทยาลัยราชภัฎสวนสุนันทา,,1,Merck,CL07-100ML,...,ศิริพรรณ สุวัฒน์วรกุลชัย (T.086-710-0829),Sales,,True,696,Antibody Technologies,Protein & Pathway Analysis,C16,LR4,Biology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40218,IVZ6608-00282,2023-08-07 00:00:00,ภาควิชาเภสัชวิทยา คณะวิทยาศาสตร์ มหาวิทยาลัยมหิดล,ZH-MU-SC-PH,ดร.นพ.พรรษกร ตันรัตนะ,คณะวิทยาศาสตร์ มหาวิทยาลัยมหิดล,,1,Supelco,SUP-C-045-1ML,...,เสาวภา ชโลทร (T.086-448-6508),Sales,,True,535,Drugs & Metrological Institutes,Reference Materials (RM),C47,LR2,Chemistry
40219,IVZ6608-00283,2023-08-07 00:00:00,ภาควิชาชีวเคมี คณะวิทยาศาสตร์ มหาวิทยาลัยมหิดล,ZH-MU-SC-BC,ดร.ณัฐวดี ปัญญาอินทร์,คณะวิทยาศาสตร์ มหาวิทยาลัยมหิดล,,1,Merck,102790.0250,...,เสาวภา ชโลทร (T.086-448-6508),Sales,,True,702,Classical Inorganics for QC,Analytical Chemistry & Solvents,C45,LR2,Chemistry
40220,IVZ6608-00283,2023-08-07 00:00:00,ภาควิชาชีวเคมี คณะวิทยาศาสตร์ มหาวิทยาลัยมหิดล,ZH-MU-SC-BC,ดร.ณัฐวดี ปัญญาอินทร์,คณะวิทยาศาสตร์ มหาวิทยาลัยมหิดล,,2,Sigma-Aldrich,SIA-T6508-100ML,...,เสาวภา ชโลทร (T.086-448-6508),Sales,,True,778,Inorganics for Research,Analytical Chemistry & Solvents,C45,LR2,Chemistry
40221,IVZ6608-00283,2023-08-07 00:00:00,ภาควิชาชีวเคมี คณะวิทยาศาสตร์ มหาวิทยาลัยมหิดล,ZH-MU-SC-BC,ดร.ณัฐวดี ปัญญาอินทร์,คณะวิทยาศาสตร์ มหาวิทยาลัยมหิดล,,3,Roche,ROC-11836170001,...,เสาวภา ชโลทร (T.086-448-6508),Sales,,True,755,Roche & Alliances,Partners & Alliances,C15,LR4,Biology


In [ ]:
'Sigma-Aldrich', 'Supelco',
    'Roche''Sigma''Millipore''Aldrich'
    'Supleco''Merck'

In [ ]:
searched['found'].value_counts()

True     21928
False     4510
Name: found, dtype: int64

In [ ]:
# Unmatched product based on the data provided from Merck.
# I suspect that the unmatched products are other product categories since we only have the categories from Merck SLS (life science)
# For example, top 3 products listed here are under industrial microbiology which is another BU in MERCK
unfound_pd= searched[~searched['found']][['รหัสสินค้า','รายละเอียดสินค้า']].value_counts().rename_axis(['product_code','description']).reset_index(name='counts')

,product_code,description,counts
0,105458.0500,"TSA-CASEIN-PEPTONE SOYMEAL-PEPTONE AGAR ""Merck...",136
1,105459.0500,"TSB-CASEIN-PEPTONE SOYMEAL-PEPTONE BROTH ""Merc...",95
2,105878.0500,MULLER-KAUFFMANN TETRATHIONATE-NOVOBIOCIN BROT...,84
3,105463.0500,PLATE-COUNT-AGAR (TRYPTONE GLUCOSE YEAST AGAR)...,69
4,107734.9025,Silica gel 60 (0.063-0.200 mm) for column chro...,68
...,...,...,...
1623,ROC-3335402001,"Protector RNase Inhibitor ""Roche"" 10000 Units",1
1624,ROC-3335399001,"Protector RNase Inhibitor ""Roche"" 2000 Units",1
1625,ROC-3115828001,"Proteinase K, recombinant, PCR Grade Solution ...",1
1626,ROC-11836153001,"complete Mini Protease Inhibitor Cocktail ""Roc...",1
